In [0]:
%matplotlib inline
import matplotlib.pyplot as plt

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os


# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


In [3]:
from google.colab import drive as mount_drive
mount_drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
EgoCNN = drive.CreateFile({'id':'1LEhCom26NzOuzeGgShzUazvZsHNn-9yu'})
EgoCNN.GetContentFile('EgoCNN.py')
from EgoCNN import EgoCNN, EgoDense, EgoMobile, EgoResNet
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.models import load_model
from keras.optimizers import SGD, Adam

import numpy as np
import h5py


Using TensorFlow backend.


In [0]:
hdf5_path = 'gdrive/My Drive/cassava/extraimages_224.hdf5'
f = h5py.File(hdf5_path)

In [6]:
"""ego_resnet = EgoMobile(in_size=(224,224,3),
                       out_size=(20,7,7),
                       batch_size=32, tbins=20)
model = ego_resnet.model"""
ego_resnet = EgoMobile(in_size=(224,224,3),
                       xrange=(-5,6),
                       yrange=(-5,6),
                       out_size=(30,11,11),
                       batch_size=32, tbins=30)
model = ego_resnet.model

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
ego_resnet.batch_size = 32
ego_resnet.lscale = 7

In [0]:
# callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=20, verbose=1)
cp_filepath = 'gdrive/My Drive/cassava/model_data/egomob.hdf5'
checkpointer = ModelCheckpoint(cp_filepath, monitor='val_loss', verbose=0,
                               save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=15, min_lr=0.0001)

In [9]:
model.compile(optimizer=Adam(lr=0.001),
              loss={'output0': 'categorical_crossentropy',
                    'output1': 'categorical_crossentropy',
                    'output2': 'categorical_crossentropy'},
              metrics=['acc'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input0 (InputLayer)             (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
input1 (InputLayer)             (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
mobilenetv2_1.00_224 (Model)    (None, 7, 7, 1280)   2257984     input0[0][0]                     
                                                                 input1[0][0]                     
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 7, 7, 2560)   0           mobilenetv2_1.00_224[1][0]       
          

In [0]:
# generators
train_generator = ego_resnet.batch_generator(f['train_img'])
val_generator = ego_resnet.batch_generator(f['val_img'])


In [11]:
if os.path.isfile(cp_filepath):
    print('Loading model')
    model = load_model(cp_filepath)
    history = model.fit_generator(train_generator,
                                  steps_per_epoch=32,
                                  epochs=300,
                                  validation_data=val_generator,
                                  validation_steps=64,
                                  callbacks=[early_stop, checkpointer, reduce_lr])
else:
    history = model.fit_generator(train_generator,
                                  steps_per_epoch=32,
                                  epochs=300,
                                  validation_data=val_generator,
                                  validation_steps=64,
                                  callbacks=[early_stop, checkpointer, reduce_lr])

Instructions for updating:
Use tf.cast instead.
Epoch 1/300
32/32 [==============================] - 59s 2s/step - loss: 7.2948 - output0_loss: 3.2069 - output1_loss: 2.0464 - output2_loss: 1.9726 - output0_acc: 0.0566 - output1_acc: 0.2109 - output2_acc: 0.2090 - val_loss: 6.6444 - val_output0_loss: 3.1225 - val_output1_loss: 1.7966 - val_output2_loss: 1.6553 - val_output0_acc: 0.0708 - val_output1_acc: 0.2441 - val_output2_acc: 0.2979
Epoch 2/300
32/32 [==============================] - 35s 1s/step - loss: 5.9689 - output0_loss: 2.9267 - output1_loss: 1.4781 - output2_loss: 1.4937 - output0_acc: 0.0938 - output1_acc: 0.3984 - output2_acc: 0.3389 - val_loss: 5.6386 - val_output0_loss: 2.9971 - val_output1_loss: 1.2443 - val_output2_loss: 1.3263 - val_output0_acc: 0.0889 - val_output1_acc: 0.4546 - val_output2_acc: 0.4097
Epoch 3/300
32/32 [==============================] - 36s 1s/step - loss: 5.1680 - output0_loss: 2.8632 - output1_loss: 1.0024 - output2_loss: 1.2308 - output0_acc: 0.

In [1]:
mets = ['output0_loss', 'output1_loss', 'output2_loss', 'val_output0_loss', 'val_output1_loss', 'val_output2_loss' ]
plt.figure(figsize=(15,7))
for met in mets:
    plt.plot(history.history[met])

plt.legend(mets)

NameError: ignored